In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/RuSportSum/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/RuSportSum/eval.csv" /content/eval.csv
!cp -r "/content/drive/MyDrive/RuSportSum/bald" "/content/bald"

In [5]:
import os
import random
from copy import deepcopy

import pandas as pd
import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [6]:
train_data = pd.read_csv("/content/train.csv")
eval_data = pd.read_csv("/content/eval.csv")

In [7]:
data_for_predict = train_data.loc[700:]
data_for_predict.to_csv('data_for_predict.csv', index=False, encoding='utf-8')

In [8]:
load_data = load_dataset("csv", data_files={'train': '/content/data_for_predict.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-211e3510de6277da/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['name_body'], examples['last_part_content'], max_length=400, truncation=True, padding="max_length")

In [11]:
tokenized_data = load_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/11764 [00:00<?, ? examples/s]

In [12]:
output = trainer.predict(tokenized_data['train'])

In [13]:
predict_labels = np.argmax(output.predictions, axis=1)

In [14]:
data_for_predict['result_match'] = predict_labels
data_for_predict

<ipython-input-14-f72a92442af4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_predict['result_match'] = predict_labels


,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body,result_match
700,870002,Россия,США,Чемпионат мира,1399916700,"Привет всем, кто не представляет своей жизни б...",Нелсон не смог добавить отскок после щелчка Тр...,1020506099,Россия – США. Василевский признан лучшим игрок...,2014-05-13 00:13:34,Вратар «Тампы» и сборной России Андрей Василев...,Россия – США. Василевский признан лучшим игрок...,68,1
701,286068,Зенит,Спартак Нальчик,Россия. Премьер-лига,1312115400,Приветствуем Всех любителей самой популярной и...,"Леандро бил справа после скидки головой, нальч...",113385918,«Зенит» произвел замену вратаря в матче с наль...,2011-07-31 17:40:02,Тренерский штаб «Зенита» произвел замену врата...,«Зенит» произвел замену вратаря в матче с наль...,64,0
702,1096715,Металлург Мг,Салават Юлаев,Кубок Гагарина,1458741600,"Привет всем, кто не представляет своей жизни б...",Элементы открытого хоккея и в этом овертайме п...,1038415924,«Магнитка» – «Салават Юлаев». Кайгородов прерв...,2016-03-23 19:09:01,Форвард «Магнитки» Алексей Кайгородов забросил...,«Магнитка» – «Салават Юлаев». Кайгородов прерв...,67,0
703,803461,Вашингтон,Бостон,НХЛ,1396110600,"Привет всем, кто не представляет своей жизни б...","Овечкин остается без шайбы в своей смене. Те, ...",159040573,НХЛ. «Вашингтон» – «Бостон» – 2:4. Гол Кузнецо...,2014-03-29 18:11:47,Сегодня в рамках очередного игрового дня регул...,НХЛ. «Вашингтон» – «Бостон» – 2:4. Гол Кузнецо...,43,1
704,955727,Суонси,Челси,Англия. Премьер-лига,1421506800,Всем привет! В рамках очередного тура английск...,"И сразу двойная у ""Челси"": Реми и Рамирес вмес...",1026535936,"Фабрегас стал первым игроком, который отдал 15...",2015-01-17 19:27:33,Полузащитник «Челси» Франсеск Фабрегас отдал 1...,"Фабрегас стал первым игроком, который отдал 15...",72,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12459,32449,Чили,Швейцария,перенесенные матчи,1277128800,"Привет! Еще чуть-чуть, и мой тезка даст свисто...","Эх, Гонсалес... Шел по центру, справа было два...",71365938,Швейцарец Бехрами удален в первом тайме матча ...,2010-06-21 18:37:37,Полузащитник сборной Швейцарии Валон Бехрами в...,Швейцарец Бехрами удален в первом тайме матча ...,76,0
12460,293545,Россия,Словакия,Чемпионат мира,1304446500,Вкус детских качелек все никак не дает покоя? ...,Польские комментаторы словно уговаривают Мариа...,94961248,Шатан сравнял счет в матче с Россией,2011-05-03 22:23:06,Форвард сборной Словакии Мирослав Шатан сравня...,Шатан сравнял счет в матче с Россией. Форвард ...,59,0
12461,1077519,Уфа,ЦСКА,Россия. Олимп Кубок России,1456840800,"Всем привет! Пришла весна, и вернулся российск...",Третья замена в ЦСКА : Аарон Олонаре вместо Ах...,1037645023,Алексей Березуцкий стал рекордсменом ЦСКА по ч...,2016-03-01 17:26:23,Защитник ЦСКА Алексей Березуцкий установил клу...,Алексей Березуцкий стал рекордсменом ЦСКА по ч...,83,0
12462,218578,Дания,Россия,Чемпионат мира,1274019300,"Привет! Так, кто там у нас на очереди? Дания! ...","Ох, не прошел пас Семина на дальнюю штангу, гд...",70695787,Сборная России вышла в четвертьфинал чемпионат...,2010-05-16 01:42:01,Сборная России после победы над Германией (3:2...,Сборная России вышла в четвертьфинал чемпионат...,87,1


In [15]:
result_data = pd.concat([train_data.loc[:699], data_for_predict, eval_data])
result_data

,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body,result_match
0,455452,Дания,Германия,Чемпионат Европы,1339958700,Всем привет и добро пожаловать на ЕВРО-2012 бе...,Бендтнер зарабатывал этот угловой с таким осте...,141499717,Ларс Бендер забил первый гол за сборную Германии,2012-06-18 00:50:25,Полузащитник «Байера» и сборной Германии Ларс ...,Ларс Бендер забил первый гол за сборную Герман...,79,1
1,476651,Бавария,Челси,Лига чемпионов,1337453100,Привет-привет! Настраиваемся? http://www.sport...,"Бить будут в ворота, за которыми мюнхенский су...",140901107,Дрогба сравнял счет в матче с «Баварией»,2012-05-20 00:34:22,Форвард «Челси» Дидье Дрогба забил гол в финал...,Дрогба сравнял счет в матче с «Баварией». Форв...,43,0
2,481837,Кубань,Спартак,Россия. Премьер-лига,1347714000,"Здравствуйте, друзья. Продолжаем следить за со...","А меняет Прудников Кулика. ""Спартак"" ответил з...",143405274,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,2012-09-15 12:57:47,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,94,1
3,1089413,СКА,Локомотив,Кубок Гагарина,1456417800,"Привет всем, кто не представляет своей жизни б...",3:0 и 2-1 в серии - СКА одержал пока что свою ...,1037500549,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",2016-02-25 22:21:05,Голкипер СКА Микко Коскинен сделал шатаут в тр...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",43,1
4,1092758,СКА,Динамо Москва,Кубок Гагарина,1457877600,"Привет всем, кто не представляет своей жизни б...",Момент у СКА! Кетов после рывка Каблукова брос...,1038079919,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,2016-03-13 20:14:38,СКА обыграл московское «Динамо» (4:0) в четвер...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,815435,СКА,Авангард,КХЛ,1393428600,"Привет всем, кто не представляет своей жизни б...",Отмечу прекрасную статистику Ружички после пер...,158241356,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,2014-02-26 19:31:03,Сегодня в Петербурге проходит матч регулярного...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,31,1
370,1013451,Барыс,Авангард,Кубок Гагарина,1425389400,Всем привет! Сегодня в Кубке Гагарина большой ...,"Хозяева, как и во втором матче, переводят игру...",1027691296,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,2015-03-04 14:39:39,Сегодня в четвертой игре серии первого раунда ...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,40,1
371,799065,Атлетик,Барселона,Испания. Ла Лига,1385928000,"Салют, амигос! ""Атлетик"" - ""Барселона"" - проти...",Чуть было сами себе не привезли хозяева! Оступ...,154776181,«Атлетик» – «Барселона». Пике проводит 150-й м...,2013-12-02 00:24:55,Защитник «Барселоны» Жерар Пике вышел в старто...,«Атлетик» – «Барселона». Пике проводит 150-й м...,71,0
372,786385,Манчестер Юнайтед,Халл,Англия. Премьер-лига,1399401900,"В последних трех матчах АПЛ ""Халл"" набрал лишь...","Феллайни совсем не в склад, не в лад. Кагава е...",1020359093,«Манчестер Юнайтед» – «Халл». Джонс был госпит...,2014-05-07 00:23:57,"Защитник «Манчестер Юнайтед» Фил Джонс, получи...",«Манчестер Юнайтед» – «Халл». Джонс был госпит...,79,0


In [16]:
result_data['result_match'].value_counts()

1    8749
0    4089
Name: result_match, dtype: int64

In [17]:
result_data.to_csv('result_data.csv', index=False, encoding='utf-8')

In [18]:
!cp "/content/result_data.csv" "/content/drive/MyDrive/RuSportSum/result_data.csv"